<a href="https://colab.research.google.com/github/Jonathan-Gilbert/Home_Sales/blob/main/Home_Sales_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required packages first
!pip install findspark
!pip install pyspark==3.4.0

# Install Java and Spark dependencies
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download and extract Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!mv spark-3.4.0-bin-hadoop3 spark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark"
os.environ["PYTHONPATH"] = "/content/spark/python/lib/py4j-0.10.9.7-src.zip:/content/spark/python"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=fb7a54a9dd9f7d9beaca0bbd343e957a05efcb2129331b6b7bc09d78841795d2
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 htt

In [4]:
# Import findspark and initialize.
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [19]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [20]:
!wget -q https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv -O /tmp/home_sales_revised.csv

In [21]:
df = spark.read.csv("/tmp/home_sales_revised.csv", header=True, inferSchema=True)

In [22]:
# 2. Create a temporary view of the DataFrame.
# Read the data into a DataFrame
# Show the first few rows to verify the data
df.show()

# 3. Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")

# 4. You can now run SQL queries on this temporary view
result = spark.sql("SELECT * FROM home_sales LIMIT 5")

# Show the result of the SQL query
result.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [33]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
result = spark.sql("""
    SELECT YEAR(date) AS year,
           ROUND(AVG(price), 2) AS avg_price_per_year
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
""")

# Show the result
result.show()


+----+------------------+
|year|avg_price_per_year|
+----+------------------+
|2019|          300263.7|
|2020|         298353.78|
|2021|         301819.44|
|2022|         296363.88|
+----+------------------+



In [29]:
# 4. Calculate the average price of homes with 3 bedrooms and 3 bathrooms,
# for each year the home was built, rounded to two decimal places.
result = spark.sql("""
    SELECT date_built AS year_built,
           ROUND(AVG(price), 2) AS avg_price_per_year_built
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY year_built
""")

# Show the result
result.show()

+----------+------------------------+
|year_built|avg_price_per_year_built|
+----------+------------------------+
|      2010|               292859.62|
|      2011|               291117.47|
|      2012|               293683.19|
|      2013|               295962.27|
|      2014|               290852.27|
|      2015|                288770.3|
|      2016|               290555.07|
|      2017|               292676.79|
+----------+------------------------+



In [30]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
result = spark.sql("""
    SELECT date_built AS year_built,
           ROUND(AVG(price), 2) AS avg_price_per_year_built
    FROM home_sales
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY year_built
""")

# Show the result
result.show()


+----------+------------------------+
|year_built|avg_price_per_year_built|
+----------+------------------------+
|      2010|               285010.22|
|      2011|               276553.81|
|      2012|               307539.97|
|      2013|               303676.79|
|      2014|               298264.72|
|      2015|               297609.97|
|      2016|                293965.1|
|      2017|               280317.58|
+----------+------------------------+



In [31]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# import time
import time

start_time = time.time()

# Execute the SQL query
result = spark.sql("""
    SELECT view,
           ROUND(AVG(price), 2) AS avg_price_per_view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

# Show the result
result.show()

# Measure and print the runtime
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|avg_price_per_view|
+----+------------------+
| 100|         1026669.5|
|  99|        1061201.42|
|  98|        1053739.33|
|  97|        1129040.15|
|  96|        1017815.92|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|        1026006.06|
|  92|         970402.55|
|  91|        1137372.73|
|  90|        1062654.16|
|  89|        1107839.15|
|  88|        1031719.35|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|        1056336.74|
|  84|        1117233.13|
|  83|        1033965.93|
|  82|         1063498.0|
|  81|        1053472.79|
+----+------------------+
only showing top 20 rows

--- 0.8432314395904541 seconds ---


In [34]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [35]:
# 8. Check if the table is cached.
print("Table cached successfully!")

Table cached successfully!


In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
spark.catalog.cacheTable("home_sales")

# Verify that the table has been cached
print("Table cached successfully!")

Table cached successfully!


In [37]:
# 11. Read the formatted parquet data.
url = "s3://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
df_parquet = spark.read.parquet(url)

# Show the first few rows of the DataFrame to verify the data
df_parquet.show()

Py4JJavaError: An error occurred while calling o126.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary

In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

